<a href="https://colab.research.google.com/github/Cirediallo/Models/blob/main/icefall_asr_librispeech_pretrained_tdnn_lstm_ctc_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Description
This notebook shows how to use a pre-trained tdnn-lstm_ctc model with [icefall](https://github.com/k2-fsa/icefall/).

## Environment setup

To use a pre-trained model with icefall, we have to install the following dependencies:

- [k2][k2], for FSA operations
- [torchaudio][audio], for reading sound files
- [kaldifeat][kaldifeat], for extracting features from a single sound
  file or multiple sound files

**NOTE**: [lhotse][lhotse] is used only in training time, for data preparation.


[k2]: https://github.com/k2-fsa/k2
[audio]: https://github.com/pytorch/audio
[kaldifeat]: https://github.com/csukuangfj/kaldifeat
[lhotse]: https://github.com/lhotse-speech/lhotse

### Install PyTorch and torchaudio

In [ ]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


Colab provides CUDA 11.0, so we choose to install torch==1.7.1+cu110.

The reason to select torch 1.7.1 is that we are going to install k2 using `pip install`, which depends on torch 1.7.1.

If you want to use a different version of PyTorch, please
refer to <https://k2.readthedocs.io/en/latest/installation/index.html> for installing k2 either from source or with `conda install`.

In [ ]:
#! pip install torch==1.7.1+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

### Install k2

In [ ]:
#! pip install k2==1.17

In [ ]:
#! pip install k2==1.17.dev20220725+cuda11.0.torch1.7.1 -f https://k2-fsa.org/nightly/index.html

Check that k2 was installed successfully:

In [ ]:
#! python3 -m k2.version

### All previous steps in one

In [2]:
!pip install torch==1.13.1
!pip install torchaudio==0.13.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.2 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.1+cu118 requires torch==2.0.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.1 requires torch==2.0.0, but you have torch

In [3]:
pip install k2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 MB 10.1 MB/s eta 0:00:00


In [4]:
! python3 -m k2.version


k2 version: 1.23.4
Build type: Release
Git SHA1: 62e404dd3f3a811d73e424199b3408e309c06e1a
Git date: Mon Jan 30 02:26:16 2023
Cuda used to build k2: 11.7
cuDNN used to build k2: 8.2.0
Python version used to build k2: 3.9
OS used to build k2: Ubuntu 18.04.6 LTS
CMake version: 3.25.1
GCC version: 7.5.0
CMAKE_CUDA_FLAGS:  -Wno-deprecated-gpu-targets   -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_35,code=sm_35  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_50,code=sm_50  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_60,code=sm_60  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_61,code=sm_61  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_70,code=sm_70  -lineinfo --expt-extended-lambda -use_fast_math -

### Install kaldifeat

In [5]:
! pip install kaldifeat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 482.4/482.4 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaldifeat: filename=kaldifeat-1.21-cp39-cp39-linux_x86_64.whl size=263530 sha256=748d4086a779ea1100ab2a93f0f8fce0abbdc389cef06bd70f4086dc029f70d6
  Stored in directory: /root/.cache/pip/wheels/d0/db/70/915de4b2e80a9aa90097d52e978ec5eb5c7393c1ee32f2618c
Successfully built kaldifeat


To check that kaldifeat was installed successfully, run

In [6]:
! python3 -c "import kaldifeat; print(kaldifeat.__version__)"

1.21


### Install icefall

icefall is a collection of Python scripts. All you need is just to
download its source code and set the `PYTHONPATH` environment variable.

In [7]:
! git clone https://github.com/k2-fsa/icefall

Cloning into 'icefall'...
remote: Enumerating objects: 11811, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 11811 (delta 70), reused 77 (delta 31), pack-reused 11657
Receiving objects: 100% (11811/11811), 13.69 MiB | 11.99 MiB/s, done.
Resolving deltas: 100% (8018/8018), done.


In [8]:
! pip install -q kaldialign

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.2 MB/s eta 0:00:00


### install lhoste

In [9]:
! pip install git+https://github.com/lhotse-speech/lhotse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/lhotse-speech/lhotse to /tmp/pip-req-build-eija9k8n
  Running command git clone --filter=blob:none --quiet https://github.com/lhotse-speech/lhotse /tmp/pip-req-build-eija9k8n
  Resolved https://github.com/lhotse-speech/lhotse to commit 76ae055653a65d82af9e8316683916413165df91
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 44.1 MB/s eta 0:00:00
  Created wheel for lhotse: filename=lhotse-1.14.0.dev0+git.76ae055.clean-py3-none-any.whl size=637237 sha256=86ab2c637f59dcf7f28f0e2c8b90e97011f1f7d5cf6cee9e734ec49835327af7
  St

In [10]:
! cd icefall && \
  ! pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.4 MB/s eta 0:00:00


In [14]:
import os

!pwd
! echo $PYTHONPATH


os.environ['PYTHONPATH'] += ":/content/icefall"

! echo $PYTHONPATH

/content
/env/python
/env/python:/content/icefall


In [15]:
#export PYTHONPATH=/content:$PYTHONPATH

## Download pre-trained Tdnn-Lstm CTC model

To make the following steps easier, we decide to download the model
to `icefall/egs/librispeech/ASR/tmp-lstm`

In [ ]:
! apt-get install -y -qq tree sox git-lfs

Selecting previously unselected package libopencore-amrnb0:amd64.
(Reading database ... 122349 files and directories currently installed.)
Preparing to unpack .../0-libopencore-amrnb0_0.1.5-1_amd64.deb ...
Unpacking libopencore-amrnb0:amd64 (0.1.5-1) ...
Selecting previously unselected package libopencore-amrwb0:amd64.
Preparing to unpack .../1-libopencore-amrwb0_0.1.5-1_amd64.deb ...
Unpacking libopencore-amrwb0:amd64 (0.1.5-1) ...
Selecting previously unselected package libsox3:amd64.
Preparing to unpack .../2-libsox3_14.4.2+git20190427-2+deb11u2build0.20.04.1_amd64.deb ...
Unpacking libsox3:amd64 (14.4.2+git20190427-2+deb11u2build0.20.04.1) ...
Selecting previously unselected package libsox-fmt-alsa:amd64.
Preparing to unpack .../3-libsox-fmt-alsa_14.4.2+git20190427-2+deb11u2build0.20.04.1_amd64.deb ...
Unpacking libsox-fmt-alsa:amd64 (14.4.2+git20190427-2+deb11u2build0.20.04.1) ...
Selecting previously unselected package libsox-fmt-base:amd64.
Preparing to unpack .../4-libsox-fmt-b

**CAUTION**: You have to run `sudo apt-get install git-lfs`
and `git lfs install` in order to download the pre-trained model.

In [ ]:
! cd icefall/egs/librispeech/ASR && rm -rf tmp-lstm

In [ ]:
! cd icefall/egs/librispeech/ASR && \
  mkdir tmp-lstm && \
  cd tmp-lstm && \
  git lfs install && \
  git clone https://huggingface.co/pkufool/icefall_asr_librispeech_tdnn-lstm_ctc && \
  cd icefall_asr_librispeech_tdnn-lstm_ctc && \
  cd ../.. && \
  tree tmp-lstm

In [ ]:
! ffprobe -show_format icefall/egs/librispeech/ASR/tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1089-134686-0001.flac

In [ ]:
! ffprobe -show_format icefall/egs/librispeech/ASR/tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0001.flac

In [ ]:
! ffprobe -show_format icefall/egs/librispeech/ASR/tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0002.flac

Due to limited memory provided by Colab, you have to upgrade to Colab Pro
to run `HLG decoding + LM rescoring`.

In [16]:
! pip install lhotse sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## HLG decoding (1best)

In [ ]:
! cd icefall/egs/librispeech/ASR && \
    PYTHONPATH=/content/icefall python3 ./tdnn_lstm_ctc/pretrained.py \
      --method 1best \
      --checkpoint ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/exp/pretrained.pt \
      --words-file ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/words.txt \
      --HLG ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/HLG.pt \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1089-134686-0001.flac \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0001.flac \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0002.flac 

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
2022-11-02 14:32:26,910 INFO [pretrained.py:162] {'feature_dim': 80, 'subsampling_factor': 3, 'num_classes': 72, 'sample_rate': 16000, 'search_beam': 20, 'output_beam': 5, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'checkpoint': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/exp/pretrained.pt', 'words_file': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/words.txt', 'HLG': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/HLG.pt', 'method': '1best', 'G': None, 'ngram_lm_scale': 0.8, 'sound_files': ['./tmp-lstm/icefall_asr_librispeech_tdnn-

## HLG decoding + LM rescoring

In [ ]:
! cd icefall/egs/librispeech/ASR && \
    PYTHONPATH=/content/icefall python3 ./tdnn_lstm_ctc/pretrained.py \
      --method whole-lattice-rescoring \
      --checkpoint ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/exp/pretrained.pt \
      --words-file ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/words.txt \
      --HLG ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/HLG.pt \
      --G ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lm/G_4_gram.pt \
      --ngram-lm-scale 0.08 \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1089-134686-0001.flac \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0001.flac \
      ./tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/test_wavs/1221-135766-0002.flac 

/usr/local/lib/python3.7/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
2022-11-02 14:35:30,881 INFO [pretrained.py:162] {'feature_dim': 80, 'subsampling_factor': 3, 'num_classes': 72, 'sample_rate': 16000, 'search_beam': 20, 'output_beam': 5, 'min_active_states': 30, 'max_active_states': 10000, 'use_double_scores': True, 'checkpoint': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/exp/pretrained.pt', 'words_file': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/words.txt', 'HLG': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lang_phone/HLG.pt', 'method': 'whole-lattice-rescoring', 'G': './tmp-lstm/icefall_asr_librispeech_tdnn-lstm_ctc/data/lm/G_4_gram.p

## Training

### Data preparation

In [17]:
! cd icefall/egs/librispeech/ASR && \
  ./prepare.sh

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
2023-04-26 12:08:47 (prepare.sh:97:main) Stage 1: Prepare LibriSpeech manifest
Dataset parts:   0% 0/7 [00:00<?, ?it/s]2023-04-26 12:08:51,475 INFO [librispeech.py:170] Processing LibriSpeech subset: train-clean-100

Distributing tasks: 0it [00:00, ?it/s]
Distributing tasks: 6it [00:00, 58.35it/s]
Distributing tasks: 12it [00:00, 49.07it/s]
Distributing tasks: 18it [00:00, 44.19it/s]
Distributing tasks: 23it [00:00, 44.87it/s]
Distributing tasks: 28it [00:00, 45.16it/s]
Distributing tasks: 33it [00:00, 44.22it/s]
Distributing tasks: 38it [00:00, 44.08it/s]
Distributing tasks: 43it [00:00, 43.32it/s]
Distributing tasks: 48it [00:01, 43.31it/s]
Distributing tasks: 53it [00:01, 43.96it/s]
Distributing tasks: 58it [00:01, 44.03it/s]
Distributing tasks: 63it [00:01, 39.34it/s]
Distributing tasks: 68it [00:01, 37.48it/s]
Distributing tasks: 73it [00:01, 38.80it/s]
Distributing tasks: 77it [00:01, 33.52it/s]
Distrib

### Launch training

In [ ]:
# https://k2-fsa.github.io/icefall/recipes/Non-streaming-ASR/librispeech/tdnn_lstm_ctc.html